# Brownian Motion

In [ ]:
using KadanoffBaym
using PyPlot
using PyCall
qt = pyimport("qutip")
np = pyimport("numpy")

PyPlot.matplotlib.rc("text", usetex=true)
PyPlot.matplotlib.rc("font", family="serif", size=16)

## KadanoffBaym.jl

In [ ]:
# struct Classical <: KadanoffBaym.GreenFunctionType end
# const Classical_{T,S} = GreenFunction{T,S,Classical}

# Base.@propagate_inbounds function Base.getindex(A::Classical_, I...)
#   Base.getindex(A.data, I...)
# end
# Base.@propagate_inbounds function Base.setindex!(A::Classical_, v, F::Vararg{Union{Int64,Colon}, 2})
#   __setindex!(A, v, (..,), F)
# end
# Base.@propagate_inbounds function Base.setindex!(A::Classical_, v, I...)
#   __setindex!(A, v, front_last2(I)...)
# end
# Base.@propagate_inbounds function __setindex!(A::Classical_, v, F, L::Tuple{F1,F2}) where {F1,F2}
#   if ==(L...)
#     setindex!(A.data, v, F..., L...)
#   else 
#     setindex!(A.data, v, F..., L...)
#     setindex!(A.data, v, F..., reverse(L)...)
#   end
# end
# Base.resize!(A::Classical_, t::Int) = Base.resize!(A, t, t)
# function Base.resize!(A::Classical_, t::Vararg{Int,2})
#   newdata = typeof(A)(zeros(eltype(A),KadanoffBaym.front2(size(A))...,t...))

#   T = min(last(size(A)), last(t))

#   for t=1:T, t′=t:T
#     @views newdata[t,t′] = A[t,t′]
#   end

#   A.data = newdata.data
#   return A
# end

In [ ]:
α = -1
D = 10

GK = GreenFunction(reshape(ComplexF64[-1.0im], 1, 1), Classical)

f_vert(t1,t2) = [+α * GK[t1,t2]]
f_diag(times,t1) = (println(times[t1]); [+2α * GK[t1,t1] - 1.0im * D])

tspan = (0.0, 4.0)
sol, _ = kbsolve(
  (_, _, t1, t2) -> f_vert(t1,t2),
  (_, times, t1) -> f_diag(times, t1), [GK], tspan[1], tspan[2],
  init_dt=1e-4, atol=1e-4, rtol=1e-4)

In [ ]:
α = -1
D = 10
GK = GreenFunction(reshape(ComplexF64[-1.0im], 1, 1), KadanoffBaym.Classical)

f_vert(t1,t2) = [+α * GK[t1,t2]]
f_diag(times,t1) = (println(times[t1]); [+2α * GK[t1,t1] - 1.0im * D])

tspan = (0.0, 4.0)
sol, _ = kbsolve(
  (_, _, t1, t2) -> f_vert(t1,t2),
  (_, times, t1) -> f_diag(times, t1), [GK], tspan,
  dtini=1e-4, atol=1e-4, rtol=1e-4)

In [ ]:
n = length(sol.t) - 1
T = sol.t[end]

## Plotting

In [ ]:
figure(figsize=(11, 3.5))
subplot(121)
plot(sol.t, [-GK.data[k, k] |> imag for k in 1:n+1])
xlim(0, abs(α) * T)
ylim(0, 5.1)
yticks([0, 1, 2, 3, 4, 5])
xlabel("\$|\\alpha|T\$")
ylabel("\$G^K(T, 0)\$")

In [ ]:
cmap = "gist_heat"
figure(figsize=(5, 4))

imshow(-GK.data |> imag, ColorMap(cmap))
colorbar(orientation="vertical")
ax = gca()
ax.set_xticks([0, n/2 +
        1, n + 1])
ax.set_xticklabels([0, abs(α) .*  T/2, abs(α) .* T])
ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")
ax.set_yticks([0, n/2 + 1, n + 1])
ax.set_yticklabels([0, abs(α) .* T/2, abs(α) .*  T])
xlabel("\$|\\alpha|t'\$")
ylabel("\$|\\alpha|t\$")
clim((0, 5))
tight_layout()